In [1]:
import sys
sys.path.append('./../src')

In [36]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import datetime
import time
import os
import random
import re
from bs4 import BeautifulSoup
import requests
# Scrape is my scraping module scrape.py
import scrape
from importlib import reload
from jikanpy import Jikan
from copy import deepcopy
from joblib import Parallel, delayed
from math import sqrt
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from tqdm import tqdm

chromedriver = "/Applications/chromedriver" # Path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

%config InlineBackend.figure_format = 'svg'
%matplotlib inline
sns.set(color_codes=True)
plt.style.use('seaborn-colorblind')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.precision', 2)

In [39]:
reload(scrape)

<module 'scrape' from './../src/scrape.py'>

## Pickling ##

In [58]:
with open('../pickles/mal_ids_top_1000_anime.pkl', 'wb') as to_write:
    pickle.dump(mal_ids_top_1000_anime, to_write)

In [91]:
with open('../pickles/mal_ids_top_1000_anime.pkl', 'rb') as read_file:
    mal_ids_top_1000_anime = pickle.load(read_file)

In [47]:
with open('../pickles/mal_user_ids_120000.pkl', 'rb') as read_file:
    mal_user_ids_120000 = pickle.load(read_file)

In [82]:
with open('../pickles/mal_user_ids_120000.pkl', 'wb') as to_write:
    pickle.dump(mal_user_ids_120000, to_write)

In [438]:
with open('../pickles/top_anime_data_1000.pkl', 'wb') as to_write:
    pickle.dump(top_anime_data_1000, to_write)

In [443]:
with open('../pickles/top_anime_data_1000_df.pkl', 'wb') as to_write:
    pickle.dump(top_anime_data_1000_df, to_write)

In [437]:
top_anime_data_1000 = (top_anime_data_200
                       + top_anime_data_200_500
                       + top_anime_data_500_750
                       + top_anime_data_750_900
                       + top_anime_data_900_1000)

In [439]:
top_anime_data_1000_df = pd.DataFrame(top_anime_data_1000)

## Storing raw anime/user data in mongoDB ##

In [6]:
with open('../pickles/top_anime_data_1000.pkl', 'rb') as f:
    top_anime_data_1000 = pickle.load(f)

In [53]:
with open('../pickles/complete_animelist.pkl', 'rb') as f:
    complete_animelist = pickle.load(f)

In [9]:
from pymongo import MongoClient
client = MongoClient()
db = client.anime

[]

In [10]:
db.anime.insert_many(top_anime_data_1000)

In [56]:
# combined_animelist contains some improperly scraped animelists, but complete_animelists contains 
# some numpy arrays that can't get stored in mongoDB 
# May want to fix this one day
db.user.insert_many(combined_animelist)

In [22]:
cursor = db.user.find({}, {'_id': 0, 'user_id': 1})

In [24]:
next(cursor)

{'user_id': 'cindia'}

## Playing with Jikan API ##

In [89]:
jikan = Jikan()

In [40]:
cowboy_bebop = jikan.anime(1)

In [62]:
type(jikan.anime(9253))

dict

In [41]:
cowboy_bebop

{'request_hash': 'request:anime:d6092f2422f084452c84555f17c7ba047e6998d3',
 'request_cached': True,
 'request_cache_expiry': -1598495124,
 'mal_id': 1,
 'url': 'https://myanimelist.net/anime/1/Cowboy_Bebop',
 'image_url': 'https://cdn.myanimelist.net/images/anime/4/19644.jpg',
 'trailer_url': 'https://www.youtube.com/embed/qig4KOK2R2g?enablejsapi=1&wmode=opaque&autoplay=1',
 'title': 'Cowboy Bebop',
 'title_english': 'Cowboy Bebop',
 'title_japanese': 'カウボーイビバップ',
 'title_synonyms': [],
 'type': 'TV',
 'source': 'Original',
 'episodes': 26,
 'status': 'Finished Airing',
 'airing': False,
 'aired': {'from': '1998-04-03T00:00:00+00:00',
  'to': '1999-04-24T00:00:00+00:00',
  'prop': {'from': {'day': 3, 'month': 4, 'year': 1998},
   'to': {'day': 24, 'month': 4, 'year': 1999}},
  'string': 'Apr 3, 1998 to Apr 24, 1999'},
 'duration': '24 min per ep',
 'rating': 'R - 17+ (violence & profanity)',
 'score': 8.79,
 'scored_by': 527450,
 'rank': 27,
 'popularity': 36,
 'members': 1064382,
 'fa

In [14]:
killuagon646 = jikan.user(username='killuagon646', request='animelist', argument='all')

In [15]:
killuagon646

{'request_hash': 'request:user:e72bba7eb603e9a01f8eec7acc5860b847463cd5',
 'request_cached': False,
 'request_cache_expiry': 3600,
 'anime': [{'mal_id': 38883,
   'title': 'Haikyuu!!: To the Top',
   'video_url': 'https://myanimelist.net/anime/38883/Haikyuu__To_the_Top/video',
   'url': 'https://myanimelist.net/anime/38883/Haikyuu__To_the_Top',
   'image_url': 'https://cdn.myanimelist.net/images/anime/1813/105367.jpg?s=af27994870941c71f53cd94e3fa0fa89',
   'type': 'TV',
   'watching_status': 1,
   'score': 9,
   'watched_episodes': 13,
   'total_episodes': 13,
   'airing_status': 2,
   'season_name': None,
   'season_year': None,
   'has_episode_video': False,
   'has_promo_video': True,
   'has_video': True,
   'is_rewatching': False,
   'tags': None,
   'rating': 'PG-13',
   'start_date': '2020-11-01T00:00:00+00:00',
   'end_date': '2020-04-04T00:00:00+00:00',
   'watch_start_date': None,
   'watch_end_date': None,
   'days': None,
   'storage': None,
   'priority': 'Low',
   'added_

In [56]:
jikan.top(type='anime', page=1)

{'request_hash': 'request:top:58399c95e55435d6ccef63eef7ce609974e4f2d5',
 'request_cached': True,
 'request_cache_expiry': 228985,
 'top': [{'mal_id': 5114,
   'rank': 1,
   'title': 'Fullmetal Alchemist: Brotherhood',
   'url': 'https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood',
   'image_url': 'https://cdn.myanimelist.net/images/anime/1223/96541.jpg?s=faffcb677a5eacd17bf761edd78bfb3f',
   'type': 'TV',
   'episodes': 64,
   'start_date': 'Apr 2009',
   'end_date': 'Jul 2010',
   'members': 1916739,
   'score': 9.22},
  {'mal_id': 9253,
   'rank': 2,
   'title': 'Steins;Gate',
   'url': 'https://myanimelist.net/anime/9253/Steins_Gate',
   'image_url': 'https://cdn.myanimelist.net/images/anime/5/73199.jpg?s=97b97d568f25a02cf5a22dda13b5371f',
   'type': 'TV',
   'episodes': 24,
   'start_date': 'Apr 2011',
   'end_date': 'Sep 2011',
   'members': 1556283,
   'score': 9.13},
  {'mal_id': 11061,
   'rank': 3,
   'title': 'Hunter x Hunter (2011)',
   'url': 'https://myan

In [10]:
top_anime_page1['top']

[{'mal_id': 5114,
  'rank': 1,
  'title': 'Fullmetal Alchemist: Brotherhood',
  'url': 'https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood',
  'image_url': 'https://cdn.myanimelist.net/images/anime/1223/96541.jpg?s=faffcb677a5eacd17bf761edd78bfb3f',
  'type': 'TV',
  'episodes': 64,
  'start_date': 'Apr 2009',
  'end_date': 'Jul 2010',
  'members': 1916739,
  'score': 9.22},
 {'mal_id': 9253,
  'rank': 2,
  'title': 'Steins;Gate',
  'url': 'https://myanimelist.net/anime/9253/Steins_Gate',
  'image_url': 'https://cdn.myanimelist.net/images/anime/5/73199.jpg?s=97b97d568f25a02cf5a22dda13b5371f',
  'type': 'TV',
  'episodes': 24,
  'start_date': 'Apr 2011',
  'end_date': 'Sep 2011',
  'members': 1556283,
  'score': 9.13},
 {'mal_id': 11061,
  'rank': 3,
  'title': 'Hunter x Hunter (2011)',
  'url': 'https://myanimelist.net/anime/11061/Hunter_x_Hunter_2011',
  'image_url': 'https://cdn.myanimelist.net/images/anime/11/33657.jpg?s=5724d8c22ae7a1dad72d8f4229ef803f',
  'type': 

In [87]:
mal_ids_top_1000_anime = get_top_anime_mal_ids(1000)

NameError: name 'get_top_anime_mal_ids' is not defined

## Scraping ##

In [288]:
for i in tqdm(range(856, 1200)):
    animelist_data_100_chunk = Parallel(n_jobs=4, verbose=5) \
        (map(delayed(ws.get_animelist_data), mal_user_ids_120000[i*100:i*100+100]))
    driver.quit()
    with open(f'../pickles/animelist_data_100_{i}.pkl', 'wb') as to_write:
        pickle.dump(animelist_data_100_chunk, to_write)
    # Pause for 3 minutes before continuing the for loop
    time.sleep(180)    

  0%|          | 0/344 [00:00<?, ?it/s][Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   17.7s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  1.4min
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:  2.2min finished
  0%|          | 1/344 [05:09<29:26:34, 309.02s/it][Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   11.6s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  1.3min
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:  2.0min finished
  1%|          | 2/344 [10:06<29:02:28, 305.70s/it][Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   14.3s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  1.3min
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:  2.0min finished
  1%|          | 3/344 [15:08<28:51:03, 304.59s/it][Parall

In [28]:
binh_ryan_user_ids = ['killuagon646', 'Hihibobo']

In [33]:
reload(scrape)

<module 'scrape' from './../src/scrape.py'>

In [37]:
animelist_binh_ryan = Parallel(n_jobs=1, verbose=5) \
    (map(delayed(scrape.get_animelist_data), binh_ryan_user_ids))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    8.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    8.7s finished


In [42]:
animelist_binh_ryan

[{'user_id': 'killuagon646',
  'animelist_url': 'https://myanimelist.net/animelist/killuagon646',
  'animelist_titles': ['Enen no Shouboutai: Ni no Shou',
   'Haikyuu!!: To the Top',
   'Ano Hi Mita Hana no Namae wo Bokutachi wa Mada Shiranai.',
   'Bakemono no Ko',
   'Boku no Hero Academia',
   'Boku no Hero Academia 2nd Season',
   'Boku no Hero Academia 3rd Season',
   'Boku no Hero Academia 4th Season',
   'Byousoku 5 Centimeter',
   'Cowboy Bebop',
   'Dr. Stone',
   'Enen no Shouboutai',
   'Fullmetal Alchemist: Brotherhood',
   'Haikyuu!!',
   'Haikyuu!! Second Season',
   'Haikyuu!!: Karasuno Koukou vs. Shiratorizawa Gakuen Koukou',
   'Hunter x Hunter (2011)',
   'Kami no Tou',
   'Kaze ga Tsuyoku Fuiteiru',
   'Kimetsu no Yaiba',
   'Kimi no Na wa.',
   'Koe no Katachi',
   'Made in Abyss',
   'Neon Genesis Evangelion',
   'Neon Genesis Evangelion: The End of Evangelion',
   'Ookami Kodomo no Ame to Yuki',
   'Ookiku Furikabutte',
   'Ookiku Furikabutte: Natsu no Taikai-hen'

In [46]:
len(animelist_binh_ryan[1]['animelist_titles'])

52

In [47]:
with open(f'../pickles/animelist_binh_ryan.pkl', 'wb') as f:
    pickle.dump(animelist_binh_ryan, f)

## Stitching anime list pickles together ##
1. For any animelist where I got the anime titles but not the scores, I need to fill in the scores with 0s. 

In [4]:
# Pickling my combined_animelist with all 120,000 records
with open(f'../pickles/combined_animelist.pkl', 'wb') as to_write:
    pickle.dump(combined_animelist, to_write)

In [340]:
# Pickling the list of user_ids I need to rescrape using the Cloud
with open(f'../pickles/user_ids_to_rescrape.pkl', 'wb') as to_write:
    pickle.dump(user_ids_to_rescrape, to_write)

In [3]:
combined_animelist = []
for i in tqdm(range(1200)):
    with open(f'../pickles/animelist_data_100_{i}.pkl', 'rb') as read_file:
        animelist_data_chunk = pickle.load(read_file)
    combined_animelist += animelist_data_chunk
len(combined_animelist)    

100%|██████████| 1200/1200 [00:02<00:00, 461.96it/s]


120000

In [315]:
animelist_without_nones = [animelist for animelist in combined_animelist_data if animelist['animelist_titles']]

In [319]:
sum([len(animelist['animelist_titles']) != len(animelist['animelist_scores']) 
     for animelist in animelist_without_nones])

90049

In [322]:
animelist_without_matching_scores = [animelist for animelist in animelist_without_nones if
                                    len(animelist['animelist_titles']) != len(animelist['animelist_scores'])]

In [323]:
animelist_without_matching_scores

[{'user_id': 'Uky',
  'animelist_url': 'https://myanimelist.net/animelist/Uky',
  'animelist_titles': ['Examurai Sengoku',
   'Eyeshield 21',
   'Gegege no Kitarou (2018)',
   'Gensoumaden Saiyuuki',
   "Gintama'",
   'Gugure! Kokkuri-san',
   'Inazuma Eleven',
   'Jigoku Sensei Nube',
   'Juuni Kokuki',
   'Karakurizoushi Ayatsuri Sakon',
   'Kiba',
   'Kyou kara Maou!',
   'Kyou kara Ore wa!!',
   'One Piece',
   'Tenshi na Konamaiki',
   'Yamada-kun to 7-nin no Majo (TV)',
   'Aa! Megami-sama! (TV)',
   'Aa! Megami-sama! (TV) Specials',
   'Ai no Kusabi',
   'Ai no Kusabi (2012)',
   'Ai Shoujo Pollyanna Story',
   'Ai Tenshi Densetsu Wedding Peach',
   'AIKa',
   'Air Gear',
   'Alps no Shoujo Heidi',
   'Amatsuki',
   "Angel's Feather",
   'Anime 80-nichikan Sekai Isshuu',
   'Ansatsu Kyoushitsu',
   'Ansatsu Kyoushitsu: Deai no Jikan',
   'Ao no Exorcist',
   'Bakuretsu Tenshi',
   'Bakuretsu Tenshi: Infinity',
   'Bakuten Shoot Beyblade',
   'Bakuten Shoot Beyblade 2002',
   'Ba

In [325]:
# Filtering for the user_ids I need to rescrape based on two conditions
user_ids_to_rescrape = [animelist['user_id'] for animelist in combined_animelist
                        if animelist['animelist_titles'] == ['${ item.anime_title }'] 
                        or not animelist['animelist_titles']] 